# CNN for collider data background identification - Step 1 - Dataset Preparation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/{user_name}/{repo_name}/blob/{branch_name}/path/to/notebook.ipynb)

In [ ]:
# LIBRERÍAS REQUERIDAS
import platform
print('python: '+platform.python_version())
import numpy as np
print('numpy: '+np.__version__)
import matplotlib
print('matplotlib: '+matplotlib.__version__)
import matplotlib.pyplot as plt
import pandas as pd
import sys

## Inicialización: Preparando el dataset

In [ ]:
##SI LOS DATOS SE TIENEN EN FORMATO CSV, CONVERTIRLOS A FORMATO H5
## Cargar el archivo CSV a un "DataFrame"
#df1 = pd.read_csv('data.csv')
## Guardarlo en formato h5
#df1.to_hdf('data.h5', key='my_data', mode='w')

In [ ]:
# CARGANDO ARCHIVO EJEMPLO
# Descargar de https://syncandshare.desy.de/index.php/s/llbX3zpLhazgPJ6/download
hdf=pd.HDFStore('test.h5', mode='r')
hdf.keys()

In [ ]:
# OBTENIENDO LOS DATOS EN UN DATAFRAME
df=hdf.get('/table')
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

### Dividiendo entre conjunto de Entrenamiento y conjunto de Prueba

In [ ]:
#Primeros 30 000 datos serán para entrenar
dftrain=df.iloc[0:30000].reset_index()
dftrain=dftrain.drop(['index'],axis=1)
#Últimos 10 000 datos serán para probar la red
dftest=df.iloc[30000:40000].reset_index()
dftest=dftest.drop(['index'],axis=1)
dfval=df.iloc[40000:50000].reset_index()
dfval=dfval.drop(['index'],axis=1)
dftrain.head()

In [ ]:
# Las etiquetas que definen señal y background
labeltr=dftrain
labeltr=np.asarray(labeltr.loc[:,'is_signal_new'])
labelte=dftest
labelte=np.asarray(labelte.loc[:,'is_signal_new'])

In [ ]:
#Guardando arreglos
np.save('labeltr',labeltr)
np.save('labelte',labelte)

### Obteniendo número de eventos background y señal

In [ ]:
# GRAFICANDO DISTRIBUCIÓN DE SEÑAL Y BACKGROUND EN HISTOGRAMA
# PARA CONJUNTO DE ENTRENAMIENTO

print('Train set', dftrain.groupby('is_signal_new').size())

clases = pd.Series(dftrain['is_signal_new']).value_counts()
clases.plot(kind = 'bar', rot=0)
plt.xticks(range(3))
plt.title("Signal (1) and background (0) jet frequencies in the training set")
plt.xlabel("Output")
plt.ylabel("Number of jets")

In [ ]:
# GRAFICANDO DISTRIBUCIÓN DE SEÑAL Y BACKGROUND EN HISTOGRAMA
# PARA CONJUNTO DE PRUEBA

print('Test set', dftest.groupby('is_signal_new').size())
clases1 = pd.Series(dftest['is_signal_new']).value_counts()
clases1.plot(kind = 'bar', rot=0)
plt.xticks(range(3))
plt.title("Signal (1) and background (0) jet frequencies in the test set")
plt.xlabel("Output")
plt.ylabel("Number of jets")

## Visualizando deposición de energía de alguno de los jets

A continuación calcularemos la energía depositada por uno de los jets en el sample

In [ ]:
aa=27681  # id del jet
E_aa=[]
px_aa=[]
py_aa=[]
pz_aa=[]
for i in range(100):
    E_aa.append(dftrain.iloc[aa][i*4])
    px_aa.append(dftrain.iloc[aa][i*4+1])
    py_aa.append(dftrain.iloc[aa][i*4+2])
    pz_aa.append(dftrain.iloc[aa][i*4+3])

In [ ]:
# GRAFICANDO

plt.rc('font', size=16) 
plt.rc('axes', labelsize=16)  
plt.rc('ytick', labelsize=12)

fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(111, projection='3d')

scat_plot =ax.scatter(px_aa,py_aa,pz_aa,c=E_aa,cmap='nipy_spectral',norm=matplotlib.colors.LogNorm(vmin=1, vmax=100, clip=False))

ax.set_title("Jet 27681 energy depositions")

ax.set_xlabel("$p_{x}$")

ax.set_ylabel("$p_{y}$")

ax.set_zlabel("$p_{z}$")

cb = plt.colorbar(scat_plot, pad=-1)

cb.set_label('Energy (GeV)')

plt.show()

## Calculando (E, /eta, /phi)

A continuación calculamos el momento total, la eta y la phi, y guardaremos el vector (E, eta, phi) en un arreglo. Acomodaremos los vectores en un arreglo de 40x40 pixeles. Elegiremos guardar el resultado para los jets con id: 8007, 8192, 20983, 27681.

Recordar que:

![Phi](phi_eq.png)

![Theta](theta_eq.png)

![Eta](eta_eq.png)

In [ ]:
# PARA SET DE ENTRENAMIENTO
n_idx=0
n_max=0
phi_max=0
w=0
A=[]
A_max=[]
n_8007=[]
n_20983=[]
phi_8007=[]
phi_20983=[]
E_8007=[]
E_20983=[]
n_27681=[]
phi_27681=[]
E_27681=[]
n_8192=[]
phi_8192=[]
E_8192=[]
for j in range(30000):
    E_max=0
    n=np.zeros(shape=100)
    phi=np.zeros(shape=100)
    En=np.zeros(shape=100)
   # if (j>5000):
    #    j=j+202000
        
    for i in range(100):
        df1 = dftrain.iloc[j:j+1,[0+4*i,1+4*i,2+4*i,3+4*i]]
        #print(df1)
        Px=df1.iloc[0][1]
        Py=df1.iloc[0][2]
        Pz=df1.iloc[0][3]
        E=df1.iloc[0][0]
        P=np.sqrt(Px**2+Py**2+Pz**2)
        if (P!=0):
            theta=np.arccos(Pz/P)
            n[i]=-np.log(np.tan(theta/2))
            if (n[i]<-2):
                n[i]=-2
            if (n[i]>2):
                n[i]=2
        else:
            n[i]=np.nan
        if (Px!=0):
            phi[i]=np.arctan(Py/Px)
        else:
            phi[i]=np.nan
        
        if (E!=0):
            En[i]=E
        else:
            En[i]=np.nan
        
        if (E>E_max):
            E_max=E
            n_idx=i
            
        if (abs(n[i])>n_max):
            n_max=abs(n[i])
        if (abs(phi[i]>phi_max)):
            phi_max=abs(phi[i])

        
    n1=n+2
    phi1=phi+(100*np.pi)/180 
    n2=np.round(n1/.1)
    phi2=np.round(phi1/(np.pi/36))
    n3=n2+(20-n2[0])
    phi3=phi2+(20-phi2[0])
    En1=np.zeros(shape=1600).reshape(40,40)
    En_max=np.zeros(shape=1600).reshape(40,40)
    
    for h in range(100):
        if (np.isnan(En[h])==False):
            g=int(phi2[h])
            k=int(n2[h])
            g1=int(phi3[h])
            k1=int(n3[h])
            if (g<40 and k<40):
               # if (En1[g][k]<En[h]):
                En1[g][k]=En1[g][k]+En[h]
            if (g1<40 and k1<40):
               # if (En1[g][k]<En[h]):
                En_max[g1][k1]=En_max[g1][k1]+En[h]
                
    if (j==8007 or j==20983 or j==27681 or j==8192):
        if (j==8007):
            n_8007=n
            phi_8007=phi
            E_8007=En
        elif (j==20983):
            n_20983=n
            phi_20983=phi
            E_20983=En
        elif (j==8192):
            n_8192=n
            phi_8192=phi
            E_8192=En
        elif (j==27681):
            n_27681=n
            phi_27681=phi
            E_27681=En
            
    A.append(En1)
    A_max.append(En_max)
Arr_max=np.asarray(A_max)
Arr=np.asarray(A)
Arr.shape

In [ ]:
# GUARDANDO ARREGLO
np.save('Datatrain',Arr)

In [ ]:
# PARA SET DE PRUEBA
B=[]
for j in range(10000):
    n=np.zeros(shape=100)
    phi=np.zeros(shape=100)
    En=np.zeros(shape=100)
   # if (j>5000):
    #    j=j+202000
        
    for i in range(100):
        df1 = dftest.iloc[j:j+1,[0+4*i,1+4*i,2+4*i,3+4*i]]
        #print(df1)
        Px=df1.iloc[0][1]
        Py=df1.iloc[0][2]
        Pz=df1.iloc[0][3]
        E=df1.iloc[0][0]
        P=np.sqrt(Px**2+Py**2+Pz**2)
        if (P!=0):
            theta=np.arccos(Pz/P)
            n[i]=-np.log(np.tan(theta/2))
            if (n[i]<-2):
                n[i]=-2
            if (n[i]>2):
                n[i]=2
        else:
            n[i]=np.nan
        if (Px!=0):
            phi[i]=np.arctan(Py/Px)
        else:
            phi[i]=np.nan
        if (E!=0):
            En[i]=E
        else:
            En[i]=np.nan
    n1=n+2
    phi1=phi+(100*np.pi)/180
    #Fitting each couple n-phi into one of our 200x200 pixels 
    n2=np.round(n1/.1)
    phi2=np.round(phi1/(np.pi/36))
    En1=np.zeros(shape=1600).reshape(40,40)
    
    for h in range(100):
        if (np.isnan(En[h])==False):
            g=int(phi2[h])
            k=int(n2[h])
            if (g<40 and k<40):
                En1[g][k]=En1[g][k]+En[h]
    
    B.append(En1)
Arrtest=np.asarray(B)
Arrtest.shape

In [ ]:
# GUARDANDO ARREGLO
np.save('Datatest',Arrtest)

## Graficando eta-phi para alguno de los jets seleccionados

In [ ]:
fig = plt.figure(figsize=(6,5))

ax = fig.add_subplot(111)

scat_plot =ax.scatter(n_8007,phi_8007,c=E_8007,cmap='nipy_spectral',norm=matplotlib.colors.LogNorm(vmin=1, vmax=100, clip=False))

ax.set_title("Jet 8007 energy depositions")

ax.set_xlabel("$\eta$")

ax.set_ylabel("$\phi$")


cb = plt.colorbar(scat_plot)

#cb.set_label('Energy (GeV)')


plt.show()

## Graficando la deposición de energía en 40x40 pixeles

In [ ]:
x = np.linspace(0,40,40)
y = np.linspace(0,40,40)
nv, phiv = np.meshgrid(x, y)

id_jet= 5098  # Elegir el jet

Enplot=Arr[id_jet]

#Enplot[np.where(Enplot==0)] = np.nan
        
plt.figure(figsize=(6,5))
plot=plt.pcolor(x,y,Enplot, cmap='jet',norm=matplotlib.colors.LogNorm(vmin=1, vmax=100, clip=False))

cbar = plt.colorbar()

plt.xlabel('$\eta$')
plt.ylabel('$\phi$')
plt.title("Jet "+str(id_jet)+" (background)")
cbar.set_label('Energy (GeV)')
print(dftrain.iloc[id_jet][805])

## Promediando background y señal

In [ ]:
backgrounds=np.zeros(shape=1600).reshape(40,40)
signals=np.zeros(shape=1600).reshape(40,40)
ba=0
si=0
for i in range(30000):
    if (dftrain.iloc[i][805]==0):
        backgrounds=backgrounds+Arr_max[i]
        ba=ba+1
    elif (dftrain.iloc[i][805]==1):
        signals=signals+Arr_max[i]
        si=si+1
backgrounds=backgrounds/ba
signals=signals/si

## Graficando señal promediada

In [ ]:
x = np.linspace(0,40,40)
y = np.linspace(0,40,40)
nv, phiv = np.meshgrid(x, y)


#Enplot[np.where(Enplot==0)] = np.nan
        
plt.figure(figsize=(6,5))
plot=plt.pcolor(x,y,signals, cmap='jet',norm=matplotlib.colors.LogNorm(vmin=1, vmax=100, clip=False))

cbar = plt.colorbar()

plt.xlabel('$\eta$')
plt.ylabel('$\phi$')
plt.title("Averaged signals")
cbar.set_label('Energy (GeV)')

## Graficando background promediado

In [ ]:
x = np.linspace(0,40,40)
y = np.linspace(0,40,40)
nv, phiv = np.meshgrid(x, y)


#Enplot[np.where(Enplot==0)] = np.nan
        
plt.figure(figsize=(6,5))
plot=plt.pcolor(x,y,backgrounds, cmap='jet',norm=matplotlib.colors.LogNorm(vmin=1, vmax=100, clip=False))

cbar = plt.colorbar()

plt.xlabel('$\eta$')
plt.ylabel('$\phi$')
plt.title("Averaged backgrounds")
cbar.set_label('Energy (GeV)')